# Metamorphic Relation to calculate bias

In [27]:
#Importing model, training data set and validation data set from the CNN_classifier provided
from cnn_classifier import model, X_train, y_train, X_val, y_val

(39209, 30, 30, 3)
Train on 31368 samples, validate on 7841 samples
31368/31368 [==============================] - 97s 3ms/sample - loss: 1.3279 - accuracy: 0.6276 - val_loss: 0.1631 - val_accuracy: 0.9552


# Detecting Bias and Fairness of our model using Google What-if tool

#### Help is taken from the notebook demo of WIT Smile Detector.ipynb to develop this code for my purpose

In [2]:
import numpy as np
import tensorflow as tf
import os
import cv2
from PIL import Image
from io import BytesIO
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [77]:
def df_to_examples(df, columns=None, images_path=''):
    examples = []
    if columns == None:
        columns = df.columns.values.tolist()
    for index, row in df.iterrows():
        example = tf.train.Example()
        for col in columns:
            if df[col].dtype is np.dtype(np.int64):
                example.features.feature[col].int64_list.value.append(int(row[col]))
            elif df[col].dtype is np.dtype(np.float64):
                example.features.feature[col].float_list.value.append(row[col])
            elif row[col] == row[col]:
                example.features.feature[col].bytes_list.value.append(row[col].encode('utf-8'))
        if images_path:
            fname = row['Path']
            with open(os.path.join(images_path, fname).replace('/','\\'), 'rb') as f:
                im = Image.open(f)
                img = cv2.imread(os.path.join(images_path, fname).replace('/','\\'))
                if img is not None:
                    im = Image.fromarray(img)
                    im = im.resize((30, 30), Image.BICUBIC)
                    buf = BytesIO()
                    im.save(buf, format= 'PNG')
                    im_bytes = buf.getvalue()
                    example.features.feature['image/encoded'].bytes_list.value.append(im_bytes)
            examples.append(example)
    return examples

def make_label_column_numeric(df, label_column, test):
    df[label_column] = np.where(test(df[label_column]), 1, 0)

In [78]:
#@title Load the csv file into pandas dataframe and process it for WIT
import pandas as pd

data = pd.read_csv(r'C:\Users\Vishal\Documents\BTP\MR codes\traffic\Train.csv')
print(data)
path = r"C:\Users\Vishal\Documents\BTP\MR codes\traffic"
examples = df_to_examples(data,images_path=path)

       Width  Height  Roi.X1  Roi.Y1  Roi.X2  Roi.Y2  ClassId  \
0         27      26       5       5      22      20       20   
1         28      27       5       6      23      22       20   
2         29      26       6       5      24      21       20   
3         28      27       5       6      23      22       20   
4         28      26       5       5      23      21       20   
...      ...     ...     ...     ...     ...     ...      ...   
39204     52      56       5       6      47      51       42   
39205     56      58       5       5      51      53       42   
39206     58      62       5       6      53      57       42   
39207     63      69       5       7      58      63       42   
39208     68      69       7       6      62      63       42   

                                 Path  
0      Train/20/00020_00000_00000.png  
1      Train/20/00020_00000_00001.png  
2      Train/20/00020_00000_00002.png  
3      Train/20/00020_00000_00003.png  
4      Train/20/000

In [79]:
def custom_predict(examples_to_infer):
    def load_byte_img(im_bytes):
        buf = BytesIO(im_bytes)
        return np.array(Image.open(buf), dtype=np.float64) / 255.

    ims = [load_byte_img(ex.features.feature['image/encoded'].bytes_list.value[0]) 
         for ex in examples_to_infer]
    preds = model.predict(np.array(ims))
    return preds


In [83]:
num_datapoints = 1000  
tool_height_in_px = 700 

# Decode an image from tf.example bytestring
def decode_image(ex):
    im_bytes = ex.features.feature['image/encoded'].bytes_list.value[0]
    im = Image.open(BytesIO(im_bytes))
    return im

# Define the custom distance function that compares the average color of images
def image_mean_distance(ex, exs, params):
    selected_im = decode_image(ex)
    mean_color = np.mean(selected_im, axis=(0,1))
    image_distances = [np.linalg.norm(mean_color - np.mean(decode_image(e), axis=(0,1))) for e in exs]
    return image_distances

# Setup the tool with the test examples and the trained classifier
config_builder = WitConfigBuilder(examples[:num_datapoints]).set_custom_predict_fn(
    custom_predict).set_custom_distance_fn(image_mean_distance)

WitWidget(config_builder, height=tool_height_in_px)

WitWidget(config={'model_type': 'classification', 'label_vocab': [], 'are_sequence_examples': False, 'inferenc…

# Bias Measurement using bootstrap algorithm

## Following are the steps followed in the algorithm

### 1. Create 3 variants of the database
### 2. While the size of the sample is less than the chosen size
####        1. Randomly select an observation from the dataset
####        2. Add it to the sample
### 3. For each bootstrap dataset
####        1. Tb is the dataset; Ub are the “out of bag” examples
####        2. Train a hypothesis hb on Tb
####        3. Test hb on each x in Ub 
### 4. Now for each (x,y) example we have many predictions h1(x),h2(x), …. so we can estimate bias as
###     average(h1(x),…,hn(x)) - y

In [52]:
import os
import random
from collections import defaultdict
from tensorflow.keras.utils import to_categorical
path = r"C:\Users\Vishal\Documents\BTP\MR codes\traffic\Train"

In [53]:
no_of_classes = len(os.listdir(path))
print(no_of_classes)
no_of_samples = 0
for i in range(no_of_classes):
    num_images = len(os.listdir(path+"//"+str(i)))
    no_of_samples = no_of_samples + num_images
    

    
d_1=[]
l_1=[]
d_2=[]
l_2=[]
d_3=[]
l_3=[]

for j in range(no_of_samples):
    category = random.randint(0,no_of_classes-1)
    new_path = path + "\\" + str(category)
    images = len(os.listdir(new_path))
    img = random.randint(0,images-1)        
    d_1.append(category)
    l_1.append(img)
    
print(len(d_1))
print(len(l_1))

for j in range(no_of_samples):
    category = random.randint(0,no_of_classes-1)
    new_path = path + "\\" + str(category)
    images = len(os.listdir(new_path))
    img = random.randint(0,images-1)        
    d_2.append(category)
    l_2.append(img)
    
for j in range(no_of_samples):
    category = random.randint(0,no_of_classes-1)
    new_path = path + "\\" + str(category)
    images = len(os.listdir(new_path))
    img = random.randint(0,images-1)        
    d_3.append(category)
    l_3.append(img)

43
39209
39209


In [62]:
p_1=[]
p_2=[]
p_3=[]

data_1=[]
data_2=[]
data_3=[]

c=0
for c in range(len(d_1)):
    images = os.listdir(path+'\\'+str(d_1[c]))
    d=0
    for img in images:
        if d!=l_1[c]:
            d=d+1
            continue
        im = cv2.imread(path+'\\'+str(d_1[c])+'\\'+img)
        im = Image.fromarray(im)
        img_arr = np.array(im.resize((30, 30), Image.BICUBIC))
        data_1.append(img_arr)
        break
    
print(len(data_1))
c=0
for c in range(len(d_2)):
    images = os.listdir(path+'\\'+str(d_2[c]))
    d=0
    for img in images:
        if d!=l_2[c]:
            d=d+1
            continue
        im = cv2.imread(path+'\\'+str(d_2[c])+'\\'+img)
        im = Image.fromarray(im)
        img_arr = np.array(im.resize((30, 30), Image.BICUBIC))
        data_2.append(img_arr)
        break

c=0        
for c in range(len(d_3)):
    images = os.listdir(path+'\\'+str(d_3[c]))
    d=0
    for img in images:
        if d!=l_3[c]:
            d=d+1
            continue
        im = cv2.imread(path+'\\'+str(d_3[c])+'\\'+img)
        im = Image.fromarray(im)
        img_arr = np.array(im.resize((30, 30), Image.BICUBIC))
        data_3.append(img_arr)
        break

model.fit(np.asarray(data_1), to_categorical(np.asarray(d_1),no_of_classes), epochs=8, verbose=1)
p_1 = model.predict(X_val)

model.fit(np.asarray(data_2), to_categorical(np.asarray(d_2),no_of_classes), epochs=8, verbose=1)
p_2 = model.predict(X_val)

model.fit(np.asarray(data_3), to_categorical(np.asarray(d_3),no_of_classes), epochs=8, verbose=1)
p_3 = model.predict(X_val)

39209
Train on 39209 samples
Epoch 1/8
39209/39209 [==============================] - 88s 2ms/sample - loss: 3.7616 - accuracy: 0.0230
Epoch 2/8
39209/39209 [==============================] - 89s 2ms/sample - loss: 3.7613 - accuracy: 0.0239
Epoch 3/8
39209/39209 [==============================] - 89s 2ms/sample - loss: 3.7613 - accuracy: 0.0227
Epoch 4/8
39209/39209 [==============================] - 89s 2ms/sample - loss: 3.7613 - accuracy: 0.0236
Epoch 5/8
39209/39209 [==============================] - 89s 2ms/sample - loss: 3.7613 - accuracy: 0.0239
Epoch 6/8
39209/39209 [==============================] - 89s 2ms/sample - loss: 3.7613 - accuracy: 0.0226
Epoch 7/8
39209/39209 [==============================] - 90s 2ms/sample - loss: 3.7613 - accuracy: 0.0234
Epoch 8/8
39209/39209 [==============================] - 95s 2ms/sample - loss: 3.7612 - accuracy: 0.0243
Train on 39209 samples
Epoch 1/8
39209/39209 [==============================] - 93s 2ms/sample - loss: 3.7614 - accuracy: 0

In [65]:
bias=0
print('Bias for each datapoint in the validation set is as follows')

for i in range(len(p_1)):
    ri = np.where(y_val[i]==np.amax(y_val))
    i1 = np.where(p_1[i]==np.amax(p_1))
    i2 = np.where(p_2[i]==np.amax(p_2))
    i3 = np.where(p_3[i]==np.amax(p_3))
    
    avg = (i1[0][0] + i2[0][0] + i3[0][0])/3
    bias = avg - ri[0][0]
    print(bias)
    


1.3333333333333321
4.333333333333332
2.333333333333332
-9.666666666666668
-13.666666666666668
4.333333333333332
-24.666666666666668
-8.666666666666668
2.333333333333332
5.333333333333332
4.333333333333332
-8.666666666666668
3.333333333333332
-0.6666666666666679
4.333333333333332
-14.666666666666668
-1.6666666666666679
12.333333333333332
11.333333333333332
12.333333333333332
9.333333333333332
-10.666666666666668
-12.666666666666668
-13.666666666666668
-17.666666666666668
-12.666666666666668
14.333333333333332
12.333333333333332
-21.666666666666668
14.333333333333332
12.333333333333332
-18.666666666666668
-2.666666666666668
15.333333333333332
7.333333333333332
-9.666666666666668
6.333333333333332
3.333333333333332
-21.666666666666668
3.333333333333332
-21.666666666666668
12.333333333333332
3.333333333333332
11.333333333333332
-13.666666666666668
-8.666666666666668
12.333333333333332
-14.666666666666668
14.333333333333332
2.333333333333332
11.333333333333332
-24.666666666666668
-10.666666

-20.666666666666668
3.333333333333332
3.333333333333332
3.333333333333332
9.333333333333332
4.333333333333332
2.333333333333332
-2.666666666666668
-1.6666666666666679
15.333333333333332
9.333333333333332
-4.666666666666668
14.333333333333332
8.333333333333332
11.333333333333332
13.333333333333332
11.333333333333332
-9.666666666666668
12.333333333333332
11.333333333333332
12.333333333333332
7.333333333333332
-21.666666666666668
14.333333333333332
14.333333333333332
2.333333333333332
-5.666666666666668
-9.666666666666668
-16.666666666666668
-21.666666666666668
-16.666666666666668
-8.666666666666668
3.333333333333332
12.333333333333332
-4.666666666666668
-0.6666666666666679
11.333333333333332
-18.666666666666668
12.333333333333332
-1.6666666666666679
9.333333333333332
-0.6666666666666679
13.333333333333332
10.333333333333332
3.333333333333332
-2.666666666666668
-18.666666666666668
8.333333333333332
3.333333333333332
6.333333333333332
-1.6666666666666679
15.333333333333332
6.33333333333333

-0.6666666666666679
-12.666666666666668
5.333333333333332
13.333333333333332
8.333333333333332
3.333333333333332
-21.666666666666668
8.333333333333332
-14.666666666666668
11.333333333333332
4.333333333333332
-15.666666666666668
9.333333333333332
-21.666666666666668
6.333333333333332
15.333333333333332
-7.666666666666668
-24.666666666666668
9.333333333333332
-21.666666666666668
3.333333333333332
-18.666666666666668
12.333333333333332
14.333333333333332
-9.666666666666668
-1.6666666666666679
15.333333333333332
-14.666666666666668
9.333333333333332
-0.6666666666666679
3.333333333333332
5.333333333333332
4.333333333333332
11.333333333333332
7.333333333333332
15.333333333333332
-8.666666666666668
-12.666666666666668
14.333333333333332
3.333333333333332
15.333333333333332
-3.666666666666668
4.333333333333332
11.333333333333332
7.333333333333332
-7.666666666666668
8.333333333333332
-16.666666666666668
4.333333333333332
-11.666666666666668
-14.666666666666668
-21.666666666666668
9.333333333333

6.333333333333332
15.333333333333332
2.333333333333332
13.333333333333332
6.333333333333332
-8.666666666666668
-8.666666666666668
7.333333333333332
7.333333333333332
3.333333333333332
14.333333333333332
7.333333333333332
3.333333333333332
7.333333333333332
2.333333333333332
11.333333333333332
13.333333333333332
-10.666666666666668
1.3333333333333321
-19.666666666666668
6.333333333333332
6.333333333333332
5.333333333333332
-1.6666666666666679
14.333333333333332
-24.666666666666668
-10.666666666666668
-23.666666666666668
12.333333333333332
14.333333333333332
-19.666666666666668
9.333333333333332
9.333333333333332
-25.666666666666668
-12.666666666666668
12.333333333333332
11.333333333333332
-21.666666666666668
-18.666666666666668
5.333333333333332
-16.666666666666668
10.333333333333332
-14.666666666666668
11.333333333333332
6.333333333333332
-20.666666666666668
8.333333333333332
9.333333333333332
13.333333333333332
4.333333333333332
13.333333333333332
10.333333333333332
11.333333333333332

-24.666666666666668
3.333333333333332
4.333333333333332
12.333333333333332
-11.666666666666668
9.333333333333332
13.333333333333332
-12.666666666666668
6.333333333333332
-8.666666666666668
11.333333333333332
-11.666666666666668
0.33333333333333215
-13.666666666666668
-1.6666666666666679
-12.666666666666668
4.333333333333332
-24.666666666666668
-16.666666666666668
4.333333333333332
6.333333333333332
3.333333333333332
3.333333333333332
10.333333333333332
5.333333333333332
13.333333333333332
11.333333333333332
14.333333333333332
4.333333333333332
15.333333333333332
16.333333333333332
-1.6666666666666679
5.333333333333332
15.333333333333332
7.333333333333332
-16.666666666666668
-9.666666666666668
3.333333333333332
4.333333333333332
10.333333333333332
6.333333333333332
-10.666666666666668
13.333333333333332
14.333333333333332
4.333333333333332
15.333333333333332
-10.666666666666668
14.333333333333332
-1.6666666666666679
-21.666666666666668
-0.6666666666666679
10.333333333333332
6.3333333333

...

11.333333333333332
-10.666666666666668
-18.666666666666668
-5.666666666666668
12.333333333333332
-0.6666666666666679
4.333333333333332
-8.666666666666668
15.333333333333332
-0.6666666666666679
13.333333333333332
-21.666666666666668
10.333333333333332
15.333333333333332
6.333333333333332
9.333333333333332
12.333333333333332
14.333333333333332
7.333333333333332
-9.666666666666668
7.333333333333332
-1.6666666666666679
-0.6666666666666679
-8.666666666666668
-21.666666666666668
8.333333333333332
9.333333333333332
6.333333333333332
-9.666666666666668
9.333333333333332
12.333333333333332
6.333333333333332
11.333333333333332
11.333333333333332
0.33333333333333215
-13.666666666666668
7.333333333333332
3.333333333333332
14.333333333333332
6.333333333333332
4.333333333333332
-16.666666666666668
7.333333333333332
12.333333333333332
4.333333333333332
-12.666666666666668
4.333333333333332
10.333333333333332
13.333333333333332
-16.666666666666668
-14.666666666666668
8.333333333333332
1.33333333333333

-16.666666666666668
1.3333333333333321
-16.666666666666668
-6.666666666666668
-25.666666666666668
-21.666666666666668
4.333333333333332
-12.666666666666668
3.333333333333332
6.333333333333332
8.333333333333332
5.333333333333332
13.333333333333332
14.333333333333332
9.333333333333332
-21.666666666666668
16.333333333333332
4.333333333333332
-14.666666666666668
4.333333333333332
5.333333333333332
9.333333333333332
9.333333333333332
0.33333333333333215
15.333333333333332
15.333333333333332
9.333333333333332
-8.666666666666668
2.333333333333332
8.333333333333332
12.333333333333332
13.333333333333332
1.3333333333333321
6.333333333333332
-7.666666666666668
1.3333333333333321
12.333333333333332
8.333333333333332
9.333333333333332
7.333333333333332
-21.666666666666668
-21.666666666666668
-12.666666666666668
8.333333333333332
-21.666666666666668
13.333333333333332
-3.666666666666668
7.333333333333332
-9.666666666666668
-5.666666666666668
11.333333333333332
-8.666666666666668
-8.666666666666668
-

14.333333333333332
-21.666666666666668
-1.6666666666666679
-3.666666666666668
-6.666666666666668
-11.666666666666668
14.333333333333332
3.333333333333332
15.333333333333332
7.333333333333332
12.333333333333332
7.333333333333332
-20.666666666666668
-9.666666666666668
-1.6666666666666679
12.333333333333332
-11.666666666666668
-23.666666666666668
2.333333333333332
14.333333333333332
-23.666666666666668
-7.666666666666668
-6.666666666666668
-1.6666666666666679
-14.666666666666668
-13.666666666666668
-1.6666666666666679
-21.666666666666668
-0.6666666666666679
15.333333333333332
3.333333333333332
-18.666666666666668
-21.666666666666668
-8.666666666666668
2.333333333333332
-23.666666666666668
-20.666666666666668
6.333333333333332
13.333333333333332
12.333333333333332
1.3333333333333321
15.333333333333332
-12.666666666666668
15.333333333333332
3.333333333333332
9.333333333333332
5.333333333333332
-9.666666666666668
12.333333333333332
-6.666666666666668
-18.666666666666668
1.3333333333333321
-1

8.333333333333332
5.333333333333332
7.333333333333332
-16.666666666666668
-21.666666666666668
-14.666666666666668
11.333333333333332
1.3333333333333321
4.333333333333332
14.333333333333332
14.333333333333332
4.333333333333332
-9.666666666666668
12.333333333333332
9.333333333333332
-5.666666666666668
12.333333333333332
3.333333333333332
-18.666666666666668
13.333333333333332
-13.666666666666668
-18.666666666666668
-21.666666666666668
12.333333333333332
9.333333333333332
15.333333333333332
-9.666666666666668
2.333333333333332
2.333333333333332
-21.666666666666668
6.333333333333332
15.333333333333332
13.333333333333332
-1.6666666666666679
-14.666666666666668
15.333333333333332
5.333333333333332
14.333333333333332
-16.666666666666668
5.333333333333332
-0.6666666666666679
15.333333333333332
-0.6666666666666679
-0.6666666666666679
6.333333333333332
7.333333333333332
-8.666666666666668
-15.666666666666668
3.333333333333332
14.333333333333332
-0.6666666666666679
4.333333333333332
3.33333333333

-21.666666666666668
14.333333333333332
5.333333333333332
-13.666666666666668
-4.666666666666668
2.333333333333332
-8.666666666666668
5.333333333333332
15.333333333333332
12.333333333333332
-1.6666666666666679
-8.666666666666668
-9.666666666666668
14.333333333333332
6.333333333333332
11.333333333333332
3.333333333333332
4.333333333333332
5.333333333333332
-21.666666666666668
5.333333333333332
15.333333333333332
15.333333333333332
8.333333333333332
-4.666666666666668
-23.666666666666668
-22.666666666666668
-0.6666666666666679
6.333333333333332
-13.666666666666668
9.333333333333332
8.333333333333332
11.333333333333332
6.333333333333332
-21.666666666666668
-3.666666666666668
2.333333333333332
9.333333333333332
-21.666666666666668
-14.666666666666668
11.333333333333332
5.333333333333332
-19.666666666666668
4.333333333333332
-1.6666666666666679
-16.666666666666668
-22.666666666666668
13.333333333333332
-12.666666666666668
-19.666666666666668
4.333333333333332
7.333333333333332
-21.6666666666